In [1]:
%matplotlib inline
import json
import urllib2
import os
from tables import open_file
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
from alleninf.api import get_probes_from_genes
from alleninf.data import get_values_at_locations
from alleninf.api import get_mni_coordinates_from_wells#
from alleninf.analysis import fixed_effects, approximate_random_effects, bayesian_random_effects

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 999

api_url   = "http://api.brain-map.org/api/v2/data/query.json"
ahba_dir  = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSM/Oct_26'


In [2]:
down_regulated_genes = { # Clardy et al. (2006). Acute and chronic effects of developmental iron deficiency 
                         # on mRNA expression patterns in the brain. Journal of Neural Transmission, 71, 173–96. 
                         # http://www.ncbi.nlm.nih.gov/pubmed/17447428
                    'THRSP'      :  'thyroid hormone responsive protein',
                    #''          :  'Rattus norvegicus Nclone10 mRNA',
                    'TF'         :  'transferrin',
                    'MAL'        :  'mal, T-cell differentiation protein',
                    'KLK6'       :  'kallikrein-related peptidase 6', 
                    'HOMER1'     :  'homer homolog 1 (Drosophila), neuronal immediate early gene',
                    'MOBP'       :  'myelin-associated oligodendrocytic basic protein',
                    'APOD'       :  'apolipoprotein D',
                    #''          :  'Rat troponin-c mRNA'
                    'MOG'        :  'myelin oligodendrocyte glycoprotein',
                    'CRYAB'      :  'crystallin, alpha B',
                    #'Af6'       :  'afadin',
                    #''           :  'Rattus norvegicus retroviral-like ovarian specific transcript 30-1 mRNA',
                    'APOC1'      :  'apolipoprotein C-I',
                    'CA2'        :  'carbonic anhydrase II',
                    'RASGRP1'    :  'RAS guanyl releasing protein 1',
                    'STMN4'      :  'stathmin-like 4',
                    #'Rnf28'      :  'ring finger protein 28',
                    'LYZ'        :  'lysozyme', 
                    #'LOC309574' :  'olfactory receptor',
                    'GSTM1'      :  'glutathione S-transferase mu 1', 
                    'CTSS'       :  'cathepsin S', 
                    'DCK'        :  'deoxycytidine kinase',
                    #''           :  'Rattus norvegicus similar to S-100 protein, alpha chain (LOC295214), mRNA',
                    #''           :  'Rat PMSG-induced ovarian mRNA, 3’sequence, N1'
                       }

iron_homeostasis_gene_set = { # http://amp.pharm.mssm.edu/Harmonizome/gene_set/Iron+Homeostasis(Mus+musculus)/Wikipathways+Pathways
                    'FTH1': 'ferritin heavy polypeptide 1',
                    'FTL' : 'ferritin light polypeptide',
                    'HFE' :'hemochromatosis',
                    'HFE2' :'hemochromatosis type 2 (juvenile)',
                    'IL1A' :'interleukin 1, alpha',
                    'IL6' :'interleukin 6',
                    'IL6R' :'interleukin 6 receptor',
                    'IREB2':'iron-responsive element binding protein 2',
                    'SLC40A1':'solute carrier family 40 (iron-regulated transporter), member 1',
                    'TF':'transferrin',
                    'TFR2':'transferrin receptor 2',
                    'TNF' : 'tumor necrosis factor',
}


DA_metabolism_gene_set = { # http://amp.pharm.mssm.edu/Harmonizome/gene_set/Dopamine+metabolism(Homo+sapiens)/Wikipathways+Pathways
                    'COMT': 'catechol-O-methyltransferase',
                    'DDC': 'dopa decarboxylase (aromatic L-amino acid decarboxylase)',
                    'MAOA': 'monoamine oxidase A',
                    'MAOB': 'monoamine oxidase B',
                    'NQO1': 'NAD(P)H dehydrogenase, quinone 1',
                    'PPP2CA': 'protein phosphatase 2, catalytic subunit, alpha isozyme',
                    'PPP2CB': 'protein phosphatase 2, catalytic subunit, beta isozyme',
                    'PRKACA': 'protein kinase, cAMP-dependent, catalytic, alpha',
                    'PRKACB': 'protein kinase, cAMP-dependent, catalytic, beta',
                    'PRKACG': 'protein kinase, cAMP-dependent, catalytic, gamma',
                    #'SOD1': 'superoxide dismutase 1, soluble',
                    'TH': 'tyrosine hydroxylase',
                    'TYR': 'tyrosinase ',
                }

glu_metabolism_gene_set =  { #http://software.broadinstitute.org/gsea/msigdb/cards/REACTOME_GLUTAMATE_NEUROTRANSMITTER_RELEASE_CYCLE
                'UNC13B' : 'unc-13 homolog B (C. elegans)',
                'RIMS1' : 'regulating synaptic membrane exocytosis 1',
                'GLS2' : 'glutaminase 2 (liver, mitochondrial)',
                'GLS' : 'glutaminase',
                'SLC38A2' : 'solute carrier family 38, member 2',
                'SLC17A7' : 'solute carrier family 17 (sodium-dependent in...',
                'RAB3A' : 'RAB3A, member RAS oncogene family',
                'SLC1A1' : 'solute carrier family 1 (neuronal/epithelial ...',
                'SLC1A6' : 'solute carrier family 1 (high affinity aspart...',
                'SLC1A7' : 'solute carrier family 1 (glutamate transporte...',
                'SNAP25' : 'synaptosomal-associated protein, 25kDa',
                'STX1A' : 'syntaxin 1A (brain)',
                'STXBP1' : 'syntaxin binding protein 1',
                'VAMP2' : 'vesicle-associated membrane protein 2 (synapt...',
                'SYT1' : 'synaptotagmin I'}

gaba_metabolism_gene_set =  { #http://software.broadinstitute.org/gsea/msigdb/cards/BIOCARTA_GABA_PATHWAY
                'GPHN': 'gephyrin',
                'GABARAP': 'GABA(A) receptor-associated protein',
                'DNM1': 'dynamin 1',
                'GABRA1': 'gamma-aminobutyric acid (GABA) A receptor, al...',
                'GABRA2': 'gamma-aminobutyric acid (GABA) A receptor, al...',
                'GABRA3': 'gamma-aminobutyric acid (GABA) A receptor, al...',
                'GABRA4': 'gamma-aminobutyric acid (GABA) A receptor, al...',
                'GABRA5': 'gamma-aminobutyric acid (GABA) A receptor, al...',
                'GABRA6': 'gamma-aminobutyric acid (GABA) A receptor, al...',
                'SRC': 'v-src sarcoma (Schmidt-Ruppin A-2) viral onco...'
                }

astrocyte_neuron_metabolic_gene_set = [ # de Leeuw, C.,etal (2015). Involvement of astrocyte metabolic coupling 
                                        # in Tourette syndrome pathogenesis. European Journal of Human Genetics, 
                                        # 23(August 2014), 1–4. https://doi.org/10.1038/ejhg.2015.22
                                       'ME1', 'ALDH5A1', 'GBE1', 'GALM', 'PYGL', 'CPS1','PFKFB3', 
                                       'PYGB', 'IDH2', 'ENO1','PPP1R1A', 'MDH2', 'CS','PYGM', 'PGM3',
                                       'PHKG1', 'SLC3A2', 'PFKFB4', 'KHK','LDHB', 'PCK2','SLC2A8', 
                                       'PGM2', 'GPT','AKR1B1', 'NANS', 'PDK4','OGDHL','DHTKD1', 
                                       'PFKM', 'PGM1', 'PC','AGL'
                                      ]

glu_synpase_gene_set = [ # Lennington, et.al (2016). Transcriptome Analysis of the Human Striatum in Tourette Syndrome. 
                        # Biological Psychiatry, 79(5), 372–382. https://doi.org/10.1016/j.biopsych.2014.07.018# 
                        # Tuquoise module Glutamatergic synapse - Homo sapiens (human)
                        'PPP3R1', 'GRIA2', 'GNAO1', 'GRIA1', 'GRIK5', 'GLUL', 'GLS', 'DLG4', 'CACNA1A', 
                        'CACNA1D', 'SHANK2', 'DLGAP1', 'GRM8', 'GRIA3', 'HOMER1', 'GRM5', 'GRIN1', 'GRM7', 
                        'GRM1', 'MAPK1', 'PLCB1', 'SLC38A1', 'GNB5', 'PRKCB', 'GNB1', 'PRKCG', 'GRM4', 'ITPR1', 
                        'SHANK3', 'PPP3CB', 'ADCY5', 'ADCY1', 'ADCY3', 'GRIK4', 'ADCY9', 'GNG7', 'GNG4', 'GRIN2A', 
                        'PRKACA', 'GNG3', 'PPP3CA', 'SLC1A6', 'GLS2'
                       ]

hse_gene_set = ['BEND5', 'C1QL2', 'CACNA1E', 'COL24A1', 'COL6A1', 'CRYM', 'KCNC3', 'KCNH4', 'LGALS1', 
                'MFGE8', 'NEFH', 'PRSS12', 'SCN3B', 'SCN4B', 'SNCG', 'SV2C', 'SYT2', 'TPBG', 'VAMP1']
